In [1]:
# importing libraries
from __future__ import division
import numpy as np#
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("whitegrid")
from scipy import stats
import pandas as pd
import itertools

%load_ext autoreload
%autoreload 2

Auxiliary functions

In [132]:
def intercala_listas(a, b):
    c = list(zip(a, b))
    return [elt for sublist in c for elt in sublist]

def format_names(names):
    names_latex = []
    hi_there = 0
    for n in names:
        if n=='lsig_c':
            hi_there+=1
            if hi_there==2:
                n += '2'
        if n[-1]=='2':
            n = n.replace('2','')
#             n = n.replace('2','^x')
        if n[0]=='s':
            n = n.replace('sig',r'$\hat{\sigma}')
            n += '$'
        elif n[:2]=='ls':
            n = n.replace('lsig',r'$\hat{\sigma}')
            n += '$'
#             n += ')$'
        elif n[:2]=='ps':
            n = ''
#             n = n.replace('psig',r'$\sigma')
#             n += '$%'
        elif n[:2]=='pl':
            n = ''
#             n = n.replace('plsig',r'$\sigma')
#             n += ')$%'
        elif n=="mu_y":
            n = n.replace('mu',r'$\hat{\mu}')
            n += '$'
        else:
            n= ""
#             n = n.replace('lpsig',r'$\sigma')
#             n += '^x)$%'
    #     print n
        names_latex.append(n)
    return names_latex

def format_short(AllTab):
    tab_short = AllTab.iloc[-8:].copy()
    tab_short.rename(index={'lsig_y2': 'log_variance', 
                                   'sig_x': 'spell_number',
                                  'lsig_c': 'constant',
                                   'lsig_e2': 'duration_dependence',
                                   'lsig_b2': 'heterogeneity',
                                   'lpsig_x': 'spell_number_percentage',
                                  'lpsig_x': 'spell_number_percentage',
                                  'lpsig_e': 'duration_dependence_percentage',
                                  'lpsig_b': 'heterogeneity_percentage'},inplace='True')
    tab_short = tab_short.append(pd.Series(1-AllTab.iloc[-3:].sum(),name='constant_percentage'))
    tab_short = tab_short.reindex(['log_variance', 'spell_number', 'constant', 'duration_dependence',
           'heterogeneity', 'constant_percentage','spell_number_percentage',
           'duration_dependence_percentage', 'heterogeneity_percentage'])
    return tab_short

def format_very_short(AllTab):
    tab_short = AllTab.iloc[-3:].copy()
    tab_short = tab_short.T
    tab_short.reset_index(level=0,inplace=True)
    tab_short['shareC'] = 1-AllTab.iloc[-3:].sum().values
    tab_short['var'] = AllTab.iloc[-8].values
    tab_short['mean_days'] = AllTab.loc['mu_y'].values
    tab_short.rename(columns={'lpsig_x': 'shareS',
                            'lpsig_e': 'shareDD',
                            'lpsig_b': 'shareH',
                             'index':'cat'},inplace='True')
    tab_short = tab_short[['shareC','shareDD','shareH','shareS','cat','var','mean_days']]
    return tab_short

def format_column_names(column_names):
    var_name = []
    type1 = []
    for name in column_names:
        split = name.split(',')
        n,t = split[0],split[1:]
        var_name.append(n)
        type1.append(t)
    return var_name,type1

def read_results(inputfile):
    table_raw = np.empty(23)
    names = []
    sigmas = []
    column_names = []
    noobs = []
    results_follow = 0
    first_row = 1
    f = open(inputfile,'r')
    for line in f:
        words = line.split()
#         print(words)
        if len(words)>1 and words[1]=="******":
            column_names.append("".join(words[2:-1]))
            results_follow=1
        elif results_follow==1 and len(words)==7:
            if first_row==1:
                names.append(words[0])
            sigmas.append(float(words[3]))
            if words[0]=='lpsig_b':
                results_follow=0
                table_raw = np.vstack((table_raw,np.array(sigmas)))
                noobs.append(int(words[2].replace(',',"")))
                sigmas = []
                first_row = 0
    f.close()
    return table_raw, names, column_names, noobs

def read_descriptive_stats(inputfile):
    table_raw = np.zeros((5,11,7))
    var_names = []
    row_n = 0
    table_n = 0
    column_names = []
    results_follow = 0
    names_done = 0
    f = open(inputfile,'r')
    for line in f:
        words = line.split()
        if len(words)>1 and words[0] == "Variable":
            results_follow=1
            if names_done == 0:
                stats_names = words[2:]       
        elif len(words)>1 and results_follow==1:
            if words[0]!='before92':
                if names_done ==0:
                    column_names.append(words[0])
                idx = 0
                for w in words[2:]: 
                    w = w.replace(",","")
                    table_raw[idx,row_n,table_n] = np.array(w)
                    idx+=1
                row_n += 1

            elif words[0]=='before92':
                if names_done ==0:
                    column_names.append(words[0])
                    names_done = 1 
                idx=0
                for w in words[2:]: 
                    w = w.replace(",","")
                    table_raw[idx,row_n,table_n] = np.array(w)
                    idx+=1
                results_follow=0
                table_n +=1
                row_n = 0
    f.close()
    column_names_format =  ['duration','duration_last','age','female','college',
                  'year_of_spell','last_T','last_P','last_A','before_84','before_92']
    table_names_format = ['Raw', 'LTU', 'STU_noAjd', 'STU','STU_Recalls', 'NE_noAjd', 'NE']
    means_table = pd.DataFrame(table_raw[1,:,:].T, columns=column_names_format) #, index=names_latex
    means_table['version'] = table_names_format
    stds_table = pd.DataFrame(table_raw[2,:,:].T, columns=column_names_format) #, index=names_latex
    stds_table['version'] = table_names_format

    final_table = pd.concat([means_table, stds_table]).sort_index(kind='merge')
    final_table['stat'] = ['mean','std']*7
    final_table = final_table[list(final_table.columns[-2:])+list(final_table.columns[:-2])]
    
    return final_table, means_table

def read_descriptive_stats_T(inputfile):
    table_raw = np.zeros((5,12,7))
    var_names = []
    row_n = 0
    table_n = 0
    column_names = []
    results_follow = 0
    names_done = 0
    f = open(inputfile,'r')
    for line in f:
        words = line.split()
        if len(words)>1 and words[0] == "Variable":
            results_follow=1
            if names_done == 0:
                stats_names = words[2:]       
        elif len(words)>1 and results_follow==1:
            if words[0]!='before92':
                if names_done ==0:
                    column_names.append(words[0])
                idx = 0
                for w in words[2:]: 
                    w = w.replace(",","")
                    table_raw[idx,row_n,table_n] = np.array(w)
                    idx+=1
                row_n += 1

            elif words[0]=='before92':
                if names_done ==0:
                    column_names.append(words[0])
                    names_done = 1 
                idx=0
                for w in words[2:]: 
                    w = w.replace(",","")
                    table_raw[idx,row_n,table_n] = np.array(w)
                    idx+=1
                results_follow=0
                table_n +=1
                row_n = 0
    f.close()
#     print(table_raw[1,:,:].T)
    column_names_format =  ['duration','duration_last','age','female','college',
                  'year_of_spell','last_T','last_P','last_A','TempContracts','before_84','before_92']
    table_names_format = ['Raw', 'LTU', 'STU_noAjd', 'STU','STU_Recalls', 'NE_noAjd', 'NE']
    means_table = pd.DataFrame(table_raw[1,:,:].T, columns=column_names_format) #, index=names_latex
    means_table['version'] = table_names_format
    stds_table = pd.DataFrame(table_raw[2,:,:].T, columns=column_names_format) #, index=names_latex
    stds_table['version'] = table_names_format

    final_table = pd.concat([means_table, stds_table]).sort_index(kind='merge')
    final_table['stat'] = ['mean','std']*7
    final_table = final_table[list(final_table.columns[-2:])+list(final_table.columns[:-2])]
    
    return final_table, means_table

## Reading the data
### New version

In [ ]:
tab_short = 
    tab_short = tab_short.T
    tab_short.reset_index(level=0,inplace=True)
    tab_short['shareC'] = 1-AllTab.iloc[-3:].sum().values
    tab_short['var'] = AllTab.iloc[-8].values
    tab_short['mean_days'] = AllTab.loc['mu_y'].values
    tab_short.rename(columns={'lpsig_x': 'shareS',
                            'lpsig_e': 'shareDD',
                            'lpsig_b': 'shareH',
                             'index':'cat'},inplace='True')
    tab_short = tab_short[['shareC','shareDD','shareH','shareS','cat','var','mean_days']]

In [133]:
table_raw, names, column_names, noobs = read_results('results/all_results96.log')
names_latex = format_names(names)
final_names = ['mu_y','sig_y','sig_c','psig_c','sig_e','psig_e','sig_b','psig_b',
                     'lsig_c','plsig_c','lsig_e','lpsig_e','lsig_b','lpsig_b']
parenthesis_names = ['psig_c','psig_e','psig_b','plsig_c','lpsig_e','lpsig_b']
# table_raw_r, names_r, column_names_r, noobs_r = read_results('results/table_recalls.log')
names[6] = 'lsig_co'
AllTab = pd.DataFrame(table_raw[1:,:].T, index=names, columns=column_names)

# rows
AllTab = AllTab.T
AllTab['plsig_c'] = AllTab['lsig_c']/AllTab['lsig_y2']
AllTab = AllTab[final_names]

AllTab['N'] = np.array(noobs)

#columns
AllTab = AllTab.T
AllTab = AllTab[['Raw','LTU','STU+SpellAdj','NE+SpellAdj']]
AllTab = AllTab.rename(columns={'STU+SpellAdj':'STU','NE+SpellAdj':"NE"})
for name in final_names:
    if name in parenthesis_names:
        AllTab.loc[name]= AllTab.loc[name].apply(lambda x: "({:,.2f})".format(float(x)))
    else:
        AllTab.loc[name]= AllTab.loc[name].apply(lambda x: "{:,.2f}".format(float(x)))
AllTab.loc['n'] = AllTab.loc['N'].apply(lambda x: "{:,.0f}".format(float(x)))

AllTab = AllTab.rename(index=dict(zip(names,names_latex)))
print(AllTab.to_latex(escape=False))

\begin{tabular}{lllll}
\toprule
{} &        Raw &         LTU &         STU &          NE \\
\midrule
$\hat{\mu}_y$    &     190.76 &      253.93 &      226.00 &      182.17 \\
$\hat{\sigma}_y$ &  66,216.70 &  137,446.50 &  171,335.50 &  142,494.00 \\
$\hat{\sigma}_c$ &  43,338.46 &   82,829.80 &   71,756.18 &   49,024.22 \\
                 &     (0.65) &      (0.60) &      (0.42) &      (0.34) \\
$\hat{\sigma}_e$ &  15,928.33 &   36,268.23 &   78,898.72 &   77,632.69 \\
                 &     (0.24) &      (0.26) &      (0.46) &      (0.54) \\
$\hat{\sigma}_b$ &   6,949.91 &   18,348.45 &   20,680.60 &   15,837.13 \\
                 &     (0.10) &      (0.13) &      (0.12) &      (0.11) \\
$\hat{\sigma}_c$ &       1.64 &        1.64 &        1.64 &        1.64 \\
                 &     (1.29) &      (1.12) &      (0.56) &      (0.49) \\
$\hat{\sigma}_e$ &      -0.62 &       -0.51 &        0.77 &        0.90 \\
                 &    (-0.51) &     (-0.36) &      (0.25) &      (0.26) \

In [151]:
table_raw_e, names, column_names_e, noobs_e = read_results('results/all_results_educ_96.log')
table_raw_g, names, column_names_g, noobs_g = read_results('results/all_results_sex_96.log')
names_latex = format_names(names)
final_names = ['mu_y','sig_y','sig_c','psig_c','sig_e','psig_e','sig_b','psig_b',
                     'lsig_c','plsig_c','lsig_e','lpsig_e','lsig_b','lpsig_b']
parenthesis_names = ['psig_c','psig_e','psig_b','plsig_c','lpsig_e','lpsig_b']
# table_raw_r, names_r, column_names_r, noobs_r = read_results('results/table_recalls.log')
names[6] = 'lsig_co'
AllTab = pd.DataFrame(table_raw_g[1:,:].T, index=names, columns=column_names_g)
Tab_e = pd.DataFrame(table_raw_e[1:,:].T, index=names, columns=column_names_e)
AllTab= pd.concat((AllTab,Tab_e),axis=1)

# rows
AllTab = AllTab.T
AllTab['plsig_c'] = AllTab['lsig_c']/AllTab['lsig_y2']
AllTab = AllTab[final_names]

AllTab['N'] = np.array(np.hstack((noobs_g,noobs_e)))

#columns
AllTab = AllTab.T
# 
AllTab = AllTab.rename(columns={'STU+SpellAdj,women':'Women','STU+SpellAdj,men':'Men',
                               'STU+SpellAdj,college':'College','STU+SpellAdj,nocollege':'Non-College'})
AllTab = AllTab[['Women','Men','College','Non-College']]
for name in final_names:
    if name in parenthesis_names:
        AllTab.loc[name]= AllTab.loc[name].apply(lambda x: "({:,.2f})".format(float(x)))
    else:
        AllTab.loc[name]= AllTab.loc[name].apply(lambda x: "{:,.2f}".format(float(x)))
AllTab.loc['N'] = AllTab.loc['N'].apply(lambda x: "{:,.0f}".format(float(x)))

AllTab = AllTab.rename(index=dict(zip(names,names_latex)))
print(AllTab.to_latex(escape=False))

\begin{tabular}{lllll}
\toprule
{} &       Women &         Men &     College & Non-College \\
\midrule
$\hat{\mu}_y$    &      284.70 &      178.19 &      197.69 &      230.83 \\
$\hat{\sigma}_y$ &  248,107.00 &  103,720.50 &  126,073.70 &  179,093.10 \\
$\hat{\sigma}_c$ &  105,323.20 &   44,418.57 &   52,585.83 &   75,164.22 \\
                 &      (0.42) &      (0.43) &      (0.42) &      (0.42) \\
$\hat{\sigma}_e$ &  118,512.80 &   46,636.29 &   59,982.81 &   82,045.23 \\
                 &      (0.48) &      (0.45) &      (0.48) &      (0.46) \\
$\hat{\sigma}_b$ &   24,270.96 &   12,665.60 &   13,505.06 &   21,883.65 \\
                 &      (0.10) &      (0.12) &      (0.11) &      (0.12) \\
$\hat{\sigma}_c$ &        1.64 &        1.64 &        1.64 &        1.64 \\
                 &      (0.56) &      (0.58) &      (0.55) &      (0.56) \\
$\hat{\sigma}_e$ &        0.88 &        0.69 &        0.88 &        0.74 \\
                 &      (0.28) &      (0.24) &      (0.28) & 

In [158]:
table_raw_bc, names, column_names_bc, noobs_bc = read_results('results/BC_all.log')
names_latex = format_names(names)
final_names = ['mu_y','sig_y','sig_c','psig_c','sig_e','psig_e','sig_b','psig_b',
                     'lsig_c','plsig_c','lsig_e','lpsig_e','lsig_b','lpsig_b']
parenthesis_names = ['psig_c','psig_e','psig_b','plsig_c','lpsig_e','lpsig_b']
names[6] = 'lsig_co'
AllTab = pd.DataFrame(table_raw_bc[1:,:].T, index=names, columns=column_names_bc)

# rows
AllTab = AllTab.T
AllTab['plsig_c'] = AllTab['lsig_c']/AllTab['lsig_y2']
AllTab = AllTab[final_names]

AllTab['N'] = np.array(noobs_bc)

#columns
AllTab = AllTab.T
# 
AllTab = AllTab.rename(columns={'STU+Spellcorrection,2002-2007':'2002-2007',
                               'STU+Spellcorrection,2008-2013':'2008-2013'})

for name in final_names:
    if name in parenthesis_names:
        AllTab.loc[name]= AllTab.loc[name].apply(lambda x: "({:,.2f})".format(float(x)))
    else:
        AllTab.loc[name]= AllTab.loc[name].apply(lambda x: "{:,.2f}".format(float(x)))
AllTab.loc['N'] = AllTab.loc['N'].apply(lambda x: "{:,.0f}".format(float(x)))

AllTab = AllTab.rename(index=dict(zip(names,names_latex)))
print(AllTab.to_latex(escape=False))

\begin{tabular}{lll}
\toprule
{} &  2002-2007 &  2008-2013 \\
\midrule
$\hat{\mu}_y$    &     165.33 &     202.94 \\
$\hat{\sigma}_y$ &  88,430.34 &  77,439.59 \\
$\hat{\sigma}_c$ &  39,608.46 &  45,858.76 \\
                 &     (0.45) &     (0.59) \\
$\hat{\sigma}_e$ &  36,545.77 &  26,904.48 \\
                 &     (0.41) &     (0.35) \\
$\hat{\sigma}_b$ &  12,276.12 &   4,676.34 \\
                 &     (0.14) &     (0.06) \\
$\hat{\sigma}_c$ &       1.64 &       1.64 \\
                 &     (0.60) &     (0.70) \\
$\hat{\sigma}_e$ &       0.57 &       0.45 \\
                 &     (0.18) &     (0.17) \\
$\hat{\sigma}_b$ &       0.54 &       0.26 \\
                 &     (0.21) &     (0.12) \\
N                &    299,426 &    326,606 \\
\bottomrule
\end{tabular}



In [165]:
table_raw_bc, names, column_names_bc, noobs_bc = read_results('results/BC_educ.log')
names_latex = format_names(names)
final_names = ['mu_y','sig_y','sig_c','psig_c','sig_e','psig_e','sig_b','psig_b',
                     'lsig_c','plsig_c','lsig_e','lpsig_e','lsig_b','lpsig_b']
parenthesis_names = ['psig_c','psig_e','psig_b','plsig_c','lpsig_e','lpsig_b']
names[6] = 'lsig_co'
AllTab = pd.DataFrame(table_raw_bc[1:,:].T, index=names, columns=column_names_bc)

# rows
AllTab = AllTab.T
AllTab['plsig_c'] = AllTab['lsig_c']/AllTab['lsig_y2']
AllTab = AllTab[final_names]

AllTab['N'] = np.array(noobs_bc)

#columns
AllTab = AllTab.T
AllTab = AllTab[['STU+Spellcorrection,2002-2007,nocollege',
               'STU+Spellcorrection,2008-2013,nocollege',
               'STU+Spellcorrection,2002-2007,college',
               'STU+Spellcorrection,2008-2013,college']]

AllTab = AllTab.rename(columns={'STU+Spellcorrection,2002-2007,college':'2002-2007',
                                'STU+Spellcorrection,2002-2007,nocollege':'2002-2007',
                               'STU+Spellcorrection,2008-2013,college':'2008-2013',
                                'STU+Spellcorrection,2008-2013,nocollege':'2008-2013'})

for name in final_names:
    if name in parenthesis_names:
        AllTab.loc[name]= AllTab.loc[name].apply(lambda x: "({:,.2f})".format(float(x)))
    else:
        AllTab.loc[name]= AllTab.loc[name].apply(lambda x: "{:,.2f}".format(float(x)))
AllTab.loc['N'] = AllTab.loc['N'].apply(lambda x: "{:,.0f}".format(float(x)))

AllTab = AllTab.rename(index=dict(zip(names,names_latex)))
print(AllTab.to_latex(escape=False))

\begin{tabular}{lllll}
\toprule
{} &  2002-2007 &  2008-2013 &  2002-2007 &  2008-2013 \\
\midrule
$\hat{\mu}_y$    &     166.84 &     207.06 &     155.12 &     179.60 \\
$\hat{\sigma}_y$ &  91,063.37 &  80,457.48 &  77,204.69 &  57,495.90 \\
$\hat{\sigma}_c$ &  40,817.05 &  47,621.09 &  32,797.27 &  36,105.39 \\
                 &     (0.45) &     (0.59) &     (0.42) &     (0.63) \\
$\hat{\sigma}_e$ &  37,264.28 &  28,090.94 &  35,669.91 &  17,541.44 \\
                 &     (0.41) &     (0.35) &     (0.46) &     (0.31) \\
$\hat{\sigma}_b$ &  12,982.03 &   4,745.46 &   8,737.51 &   3,849.07 \\
                 &     (0.14) &     (0.06) &     (0.11) &     (0.07) \\
$\hat{\sigma}_c$ &       1.64 &       1.64 &       1.64 &       1.64 \\
                 &     (0.60) &     (0.71) &     (0.58) &     (0.67) \\
$\hat{\sigma}_e$ &       0.53 &       0.42 &       0.73 &       0.51 \\
                 &     (0.17) &     (0.17) &     (0.23) &     (0.20) \\
$\hat{\sigma}_b$ &       0.55 &      